In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import re

In [68]:
DATA_DIR = "/mnt/c/Users/Admin/OneDrive - Lebanese American University/Documents/internship/WebScraping/data/"

In [50]:
url = "https://www.mushroomexpert.com/"
response= requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
table = soup.find(lambda tag: tag.name == 'table' and tag.find('i') and tag.find('i').string == 'Clavulinopsis')

In [51]:
species=table.find_all('a')
number_of_species = len(species)
print("Number of species is:", number_of_species)

Number of species is: 4


In [69]:
df=pd.DataFrame(columns=['SpeciesName','Ecology','Odor and Taste'])

In [70]:

for specie in species:
    result=[]
    url = 'https://www.mushroomexpert.com/'+specie['href']
    response=requests.get(url)
    soup=BeautifulSoup(response.content,'html.parser')
    
    name= soup.find('font', {'size':'+0'}).text
    result.append(name)

    wanted_tag = soup.find(lambda tag: tag.name == 'p' and re.search('saprobic|mycorrhizal|parasitic', tag.text, re.IGNORECASE))
    ecology=wanted_tag.a.text.strip()
    result.append(ecology)

    description = soup.find(lambda tag: tag.name=='p' and tag.find('b') and tag.find('b').string == 'Description:')
    wanted_tag= description.find_next_sibling(lambda tag: tag.name=='p'and tag.find('a') and re.search('odor|taste', tag.text, re.IGNORECASE))
    odor_taste = wanted_tag.a.next_sibling.text.strip(": ")
    result.append(odor_taste)

    df.loc[len(df)]=result

df.to_csv(DATA_DIR +'Clavulinopsis(MushroomExpert).csv', index=False)